In [ ]:
pip install geemap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.8 MB/s eta 0:00:00


In [ ]:
import ee
import geemap

In [ ]:
ee.Authenticate()
ee.Initialize(project='eealifia-21110010')

Radius

In [ ]:
#Tentukan titik koordinat
latitude = -7.4305508565781775
longitude = 109.25061577492862

#Buat objek geometri titik
point = ee.Geometry.Point([longitude, latitude])

#Buat lingkaran (buffer) dengan jar-jari 5 km
buffered_point = point.buffer(10000)

#Inisialisasi peta interaktif
Map = geemap.Map()
Map.centerObject(point, 10) #Pusatkan peta pada titik koordinat
Map.addLayer(buffered_point, {'color':'FF0000'}, 'Radius 5 km')
Map.addLayer(point, {'color' : '00FF00'}, 'Titik Koordinat')
Map

Map(center=[-7.430550856578179, 109.25061577492863], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers

    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)

    a = np.sin(dlat / 2) * np.sin(dlat / 2) + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) * np.sin(dlon / 2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    distance = R * c  # Distance in kilometers
    return distance

def recommend_restaurant(user_location, restaurants_data):
    restaurants_data['distance'] = restaurants_data.apply(
        lambda row: haversine(user_location['latitude'], user_location['longitude'], row['latitude'], row['longitude']), axis=1
    )

    nearest_restaurant = restaurants_data.loc[restaurants_data['distance'].idxmin()]

    return nearest_restaurant

# Contoh data restoran
restaurants = {
    'latitude': [37.7749, 37.7748, 37.7750, 37.7760],
    'longitude': [-122.4194, -122.4189, -122.4200, -122.4185],
    'nama_restoran': ['Resto A', 'Resto B', 'Resto C', 'Resto D'],
}

restaurants_df = pd.DataFrame(restaurants)

# Lokasi pengguna
user_location = {'latitude': 37.7755, 'longitude': -122.4184}

# Rekomendasi restoran terdekat
recommended_restaurant = recommend_restaurant(user_location, restaurants_df)

print("Restoran Terdekat:")
print("Nama: ", recommended_restaurant['nama_restoran'])
print("Latitude: ", recommended_restaurant['latitude'])
print("Longitude: ", recommended_restaurant['longitude'])
print("Jarak: ", recommended_restaurant['distance'], " km")

Restoran Terdekat:
Nama:  Resto D
Latitude:  37.776
Longitude:  -122.4185
Jarak:  0.05628787226638373  km


Radius dan Beberapa Titik Lokasi

In [ ]:
!pip install folium

In [ ]:
import folium
import pandas as pd

In [ ]:
#Fungsi untuk menambahkan marker pada peta folium
def add_marker(map_obj, location, popup_text):
  marker = folium.Marker(location = location, popup = popup_text)
  marker.add_to(map_obj)

#Baca data dari file CSV
file_path = 'dataset_geo.xlsx'
data = pd.read_excel(file_path)

In [ ]:
#Buat peta folium
map_obj = folium.Map(location=[-6.2088, 106.8456], zoom_start=13)

#Fungsi untuk menambahkan marker ousat centroid pada peta folium
def add_marker_red(map_obj, location, popup_text, icon_color = 'red'):
  marker = folium.Marker(location=location, popup=popup_text, icon=folium.Icon(color=icon_color))
  marker.add_to(map_obj)

#Tambahkan marker lokasi pusat (centroid Jakarta)
add_marker_red(map_obj,(-6.2088, 106.8456), 'LOKASI CENTROID')

#Tambahkan marker untuk setiap lokasi dari data CSV
Dataset_Baru=[]
for index, row in data.iterrows():
  location = [row['Latitude'], row['Longitude']]
  popup_text = f"Label: {row['Nama Tempat']}"
  add_marker(map_obj, location, popup_text)

In [ ]:
#Membuat Radius
point = ee.Geometry.Point(latitude, longitude)

#Buat lingkaran (buffer) dengan radius 5 km
buffered_point = point.buffer(10000) #Buffer dengan jari- jari 5 km

#Set lokasi pusat peta pada titik koordinat
map_obj.location = [latitude, longitude]

#Tambahkan lingkaran ke peta
folium.Circle(location=[latitude, longitude],
              radius = 10000,
              color = 'FF0000',
              fill = True,
              fill_color ='FF0000').add_to(map_obj)

#Tampilkan peta
map_obj

In [ ]:
#Fungsi menghitung jarak dengan Haversine
import math
def haversine_distance(lat, lon1, lat2, lon2):
  #Radius Bumi dalam kilometer
  R = 10000

  #Konversi lintang dan bujur dari derajat ke radian
  lat1_rad = math.radians(lat1)
  lon1_rad = math.radians(lon1)
  lat2_rad = math.radians(lat2)
  lon2_rad = math.radians(lon2)

  #Perbedaan koordinat
  dlat = lat2_rad - lat1_rad
  dlon = lon2_rad - lon1_rad

  #Rumus Haversine
  a = math.sin(dlat/2)**2 + math.cos(lat1_rad)*math.cos(lat2_rad)*math.sin(dlon/2)**2
  c = 2*math.atan2(math.sqrt(a), math.sqrt(1-a))

  #Hitung jarak
  distance = R*c

  return distance

In [ ]:
#data dalam Radius
data_rad = []
lat1 = latitude
lon1 = longitude
for index, row in data.iterrows():
  location = [row['Latitude'], row['Longitude']]
  jarak_km = haversine_distance(lat1, lon1, row['Latitude'], row['Longitude'])
  nama_tempat = f"{row['Nama Tempat']}"
  biaya = f"{row['Harga']}"
  rating = f"{row['Rating']}"
  if jarak_km <= 10000:
    #append a dictionary containing 'location' and 'popup_text' to data_rad
    data_rad.append({'Koordinat' : location, 'Nama Lokasi' : nama_tempat, 'Biaya' : biaya, 'Rating': rating})

#Tampilkan data_rad
data_rad = pd.DataFrame(data_rad)
data_rad

,Koordinat,Nama Lokasi,Biaya,Rating
0,"[-7.41725585789748, 109.249678699999]",Ayam Geprek Juara - GOR Purwokerto,20000,4.5
1,"[-7.42724660723575, 109.253371471163]",Spesial Ayam Geprek Mbah Kakung,14500,4.3
2,"[-7.42404009990653, 109.240742884655]",Ayam Geprek Bae,21500,4.3
3,"[-7.40937293654178, 109.240628942327]",Ayam Geprek Hara Chicken,14000,4.6
4,"[-7.39400154723343, 109.2469468]",ayam geprek zf food,18000,4.9
...,...,...,...,...
106,"[-7.42612896528449, 109.248997381032]",Rumah Makan Ayam Goreng GOAR,14000,0.0
107,"[-7.42880943333143, 109.240681736856]",Ayam Bakar & Goreng PM Maharani,15000,4.3
108,"[-7.41870123898486, 109.227258709868]",Warung tenda ayam kremes banyumas,16000,0.0
109,"[-7.42414114808505, 109.242367396374]",AYAM GORENG KALASAN BANG GIRI,20000,0.0


In [ ]:
df = pd.DataFrame(data_rad)

#Cek numerik nilai Biaya dan rating
if 'Harga' in df.columns and 'Rating' in df.columns:
  #Converting Biaya dan rating ke numerik
  df['Harga'] = pd.to_numeric(df['Harga'], errors = 'coerce')
  df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')

  #Cek kesalahan konversi
  if df[['Harga', 'Rating']]. isnull().values.any():
    print("Warning : Kolom Biaya dan rating bukan numerik")

  #Normalization
  df['Normalized Harga']=df['Harga'].min()/df['Harga'] #Cost
  df['Normalized Rating'] = df['Rating']/df['Rating'].max()

  #Weight/ Pembobotan atribut Biaya dan Rating
  bobot_harga = 0.35
  bobot_rating = 0.65

  #Perhitungan total bobot prioritas (Sum of Product)
  df['Nilai Terbobot'] = bobot_harga * df['Normalized Harga'] + bobot_rating * df['Normalized Rating']

  #Pilih data terbaik
  data_terbaik = df[df['Nilai Terbobot']== df['Nilai Terbobot'].max()]
  print('Rekomendasi terbaik :')
  print(data_terbaik)
else :
  print("'Harga' atau 'Rating' tidak ditemukan.")

'Harga' atau 'Rating' tidak ditemukan.


In [ ]:
def add_marker_green(map_obj, location, popup_text):
  marker = folium.Marker(location=location, popup=popup_text, icon = folium.Icon(color='green'))
  marker.add_to(map_obj)

#Tambahkan marker lokasi pusat (centroid Jakarta)
location = (-6.221339, 106.868167)
add_marker_green(map_obj, location, 'LOKASI TARGET')
map_obj

In [ ]:
#Tampilkan jarak titik lokasi centroid dengan titik lokasi target
jarak_hasil = haversine_distance(latitude, longitude, location[0], location[1])
print('Jarak lokasi centroid dengan lokasi target = ', jarak_hasil)

Jarak lokasi centroid dengan lokasi target =  463.6747734649015
